In [3]:
import torch
from diffusers import StableDiffusionPipeline, UNet2DConditionModel
from peft import LoraConfig, get_peft_model
import gradio as gr

# Константы
MODEL_ID = "stabilityai/stable-diffusion-2-1-base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LORA_PATH = "lora_ghibli"

pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, torch_dtype=torch.float16).to(DEVICE)

def setup_lora():
    base_unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet").to(DEVICE)

    lora_config = LoraConfig(
        r=8,
        lora_alpha=32,
        target_modules=["to_q", "to_k", "to_v", "to_out.0"],
        lora_dropout=0.1,
        bias="none"
    )

    unet_with_lora = get_peft_model(base_unet, lora_config)
    unet_with_lora.load_adapter(LORA_PATH, "default")
    pipe.unet = unet_with_lora

    pipe.unet = pipe.unet.to(dtype=torch.float16)
    pipe.text_encoder = pipe.text_encoder.to(dtype=torch.float16)
    pipe.vae = pipe.vae.to(dtype=torch.float16)

    print("Модель готова к генерации")

def generate_image(prompt, lora_scale=0.6):
    try:
        pipe.fuse_lora(lora_scale=lora_scale)
    except Exception as e:
        print(f"Ошибка при слиянии LoRA: {e}")
        raise

    with torch.autocast("cuda"):
        image = pipe(
            prompt,
            num_inference_steps=40,  # Фиксированное значение
            guidance_scale=7.5        # Фиксированное значение
        ).images[0]
    return image

setup_lora()

with gr.Blocks(title="Генератор изображений с LoRA") as demo:
    gr.Markdown("# Генератор изображений в стиле Ghibli")
    gr.Markdown("Для лучшей генерации в конце напиши 'Ghibli style'")
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(
                label="Prompt",
                placeholder="тут должен быть ваш текст...",
                lines=3
            )
            lora_scale = gr.Slider(0.1, 1.0, value=0.6, label="LoRA Scale (влияние стиля Ghibli)")
            submit_btn = gr.Button("Сгенерировать", variant="primary")

        with gr.Column():
            output_image = gr.Image(label="Результат", type="pil")

    # Примеры промптов (без steps и guidance)
    examples = gr.Examples(
        examples=[
            ["A peaceful forest scene, Ghibli style"],
            ["A flying castle with dragons, Ghibli style"],
            ["The boy with the flowers in the style of Studio Ghibli"]
        ],
        inputs=[prompt]
    )

    submit_btn.click(
        fn=generate_image,
        inputs=[prompt, lora_scale],
        outputs=output_image
    )

# Запуск интерфейса
if __name__ == "__main__":
    demo.launch(share=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/911 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Модель готова к генерации
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dd2dffcb96a57d3b27.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
